In [ ]:

!pip install -U bitsandbytes langchain huggingface_hub chromadb torch transformers yfinance langchain_community sentence_transformers

In [ ]:
import pandas as pd
from langchain.docstore.document import Document

# Read data from the CSV file (mutual fund data)
data_df = pd.read_csv('/content/comprehensive_mutual_funds_data.csv')

# Create documents for LangChain
documents = []
for _, row in data_df.iterrows():
    content = f"Scheme Name: {row['scheme_name']}, Min SIP: {row['min_sip']}, Min Lumpsum: {row['min_lumpsum']}, " \
              f"Expense Ratio: {row['expense_ratio']}%, Fund Size: {row['fund_size_cr']} Cr, Fund Age: {row['fund_age_yr']} years, " \
              f"Fund Manager: {row['fund_manager']}, Sortino: {row['sortino']}, Alpha: {row['alpha']}, SD: {row['sd']}, " \
              f"Beta: {row['beta']}, Sharpe: {row['sharpe']}, Risk Level: {row['risk_level']}, AMC: {row['amc_name']}, " \
              f"Rating: {row['rating']}, Category: {row['category']}, Sub-Category: {row['sub_category']}, " \
              f"1 Yr Return: {row['returns_1yr']}%, 3 Yr Return: {row['returns_3yr']}%, 5 Yr Return: {row['returns_5yr']}%"

    documents.append(Document(page_content=content))

# Optionally, you can display the first document to verify
print(documents[0].page_content)

Scheme Name: Aditya Birla SL Active Debt Multi-Mgr FoF-Dir Growth, Min SIP: 100, Min Lumpsum: 100, Expense Ratio: 0.27%, Fund Size: 10.0 Cr, Fund Age: 10 years, Fund Manager: Kaustubh Gupta, Sortino: 0.32, Alpha: 2.24, SD: 9.39, Beta: 0.01, Sharpe: 0.24, Risk Level: 3, AMC: Aditya Birla Sun Life Mutual Fund, Rating: 3, Category: Other, Sub-Category: FoFs Domestic, 1 Yr Return: 4.0%, 3 Yr Return: 6.5%, 5 Yr Return: 6.9%


In [ ]:
# Use Hugging Face Embeddings and ChromaDB
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

hg_embeddings = HuggingFaceEmbeddings()
persist_directory = '/content/'

# Initialize ChromaDB
langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="mutual_fund_optimization",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

<ipython-input-3-61838da741d9>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hg_embeddings = HuggingFaceEmbeddings()
<ipython-input-3-61838da741d9>:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hu

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from torch import cuda, bfloat16, float16
import transformers
from transformers import AutoTokenizer
from langchain.llms import HuggingFacePipeline

model_id = 'HuggingFaceH4/zephyr-7b-beta'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# Configure model for efficient GPU memory usage
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# Initialize the query pipeline
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=float16,
    max_length=6000,
    device_map="auto",
)

llm = HuggingFacePipeline(pipeline=query_pipeline)

Device set to use cuda:0
<ipython-input-5-494909aeff40>:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
import pandas as pd

# Define the prompt template for mutual fund data (with a focus on performance and risk)

template = """
You are an AI-professional investment report generation expert. Based on the question asked by the client, look up the provided mutual fund data, and answer the following in a user-friendly, readable format using Times New Roman:

Client-Specific: Based on the question asked, extract detailed client-specific and compliant investment report content from the provided mutual fund data. Compare the mutual fund against well-known benchmarks and mention the names of those benchmarks. Also, include the relevant disclosures without repeating the same content across sections.

Investment-Specific: Using the provided mutual fund data, extract and present investment plans suitable for the client’s profile. Ensure compliance by comparing the mutual fund’s performance with relevant benchmarks, mentioning those benchmarks, and incorporating the necessary disclosures, ensuring no content repetition.

Recommendations: Provide tailored recommendations for different client profiles and investment goals based on the client’s question. Compare the mutual fund against appropriate benchmarks and mention those benchmarks, while ensuring that necessary disclosures are included without redundancy.

Accuracy and Factual Consistency: Provide the accuracy percentage and factual consistency of the AI-generated content. Cite the sources used in the report along with links (if applicable).

Mutual Fund Information: {question}
Context: {context}
Answer:
"""




PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Set up retriever (assuming 'langchain_chroma' is your vector store and is already initialized)
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 5})

# Function to remove duplicates from retrieved documents
def remove_duplicates(documents):
    seen = set()
    unique_docs = []
    for doc in documents:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    return unique_docs

# Set up the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def get_optimized_recommendations(query):
    # Retrieve documents based on the query
    raw_docs = retriever.get_relevant_documents(query)

    # Remove duplicates
    unique_docs = remove_duplicates(raw_docs)

    # Prepare the context from the documents
    context = " ".join([doc.page_content for doc in unique_docs])

    # Use the QA chain to get the response
    result = qa_chain({"context": context, "query": query})
    return result

# Example query
query = input("Please enter your query: ")
response = get_optimized_recommendations(query)
print(response)


Please enter your query: can i invest in aditya birla with an investment of 5000 Rs currently


<ipython-input-6-9d2cc8420f1c>:51: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_docs = retriever.get_relevant_documents(query)
<ipython-input-6-9d2cc8420f1c>:60: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"context": context, "query": query})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'context': 'Scheme Name: Aditya Birla SL Inter Equity – Plan A-Direct Growth, Min SIP: 1000, Min Lumpsum: 1000, Expense Ratio: 1.75%, Fund Size: 108.0 Cr, Fund Age: 10 years, Fund Manager: Vinod Narayan Bhat, Sortino: 1.28, Alpha: -, SD: 13.99, Beta: -, Sharpe: 0.81, Risk Level: 6, AMC: Aditya Birla Sun Life Mutual Fund, Rating: 0, Category: Equity, Sub-Category: Sectoral / Thematic Mutual Funds, 1 Yr Return: 0.9%, 3 Yr Return: 17.7%, 5 Yr Return: 10.4% Scheme Name: Aditya Birla SL Inter Equity – Plan B-Direct Growth, Min SIP: 1000, Min Lumpsum: 1000, Expense Ratio: 2.15%, Fund Size: 90.0 Cr, Fund Age: 10 years, Fund Manager: Vinod Narayan Bhat, Sortino: 2.45, Alpha: -1.93, SD: 16.3, Beta: 0.87, Sharpe: 1.14, Risk Level: 6, AMC: Aditya Birla Sun Life Mutual Fund, Rating: 0, Category: Equity, Sub-Category: Sectoral / Thematic Mutual Funds, 1 Yr Return: -4.1%, 3 Yr Return: 22.7%, 5 Yr Return: 7.6% Scheme Name: Aditya Birla SL Tax Relief ’96 – Direct Growth, Min SIP: 500, Min Lumpsum: 50

In [ ]:
print(response['result'][response['result'].find("Answer:"):])



Answer:

Client-Specific:

Based on the question asked, we will analyze the performance of the Aditya Birla SL Inter Equity – Plan A-Direct Growth mutual fund for a client with a moderate risk tolerance and a time horizon of 5 years. We will compare its performance against well-known benchmarks and provide relevant disclosures without repetition.

The Aditya Birla SL Inter Equity – Plan A-Direct Growth mutual fund has an expense ratio of 1.75% and a minimum SIP and lumpsum investment amount of Rs. 1,000. The fund has been in existence for 10 years, and its fund manager is Vinod Narayan Bhat. The Sortino ratio is 1.28, indicating that the mutual fund has generated a higher return for the level of risk taken. The Alpha value of - indicates that the mutual fund has underperformed the benchmark. The Sharpe ratio of 0.81 suggests that the mutual fund has generated a higher return for the level of risk taken. The Risk Level is 6, indicating a moderate risk level.

The mutual fund's performan

In [ ]:
print(response['result'][response['result'].find("Answer:"):])



Answer:

Based on the provided mutual fund data, we recommend investing in Kotak Infrastructure & Ecoc. Reform-SP-DirGrowth over the other two funds for the following reasons:

1. Higher returns: Kotak Infrastructure & Ecoc. Reform-SP-DirGrowth has consistently outperformed the other two funds in terms of returns over different time horizons. In the past year, it has delivered a return of 23.2%, while ICICI Pru Banking & Financial Services – DirectGrowth and LIC MF Banking & Fina Serv Fund have delivered returns of 7.0% and 13.9%, respectively. Over a three-year period, Kotak Infrastructure & Ecoc. Reform-SP-DirGrowth has delivered a return of 39.7%, while ICICI Pru Banking & Financial Services – DirectGrowth and LIC MF Banking & Fina Serv Fund have delivered returns of 24.9% and 22.7%, respectively.

2. Lower risk: Despite delivering higher returns, Kotak Infrastructure & Ecoc. Reform-SP-DirGrowth has a lower risk level, as measured by its Sortino ratio and Beta. Its Sortino ratio is 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
import pandas as pd

# Define the prompt template for mutual fund data (with a focus on performance and risk)

template = """
You are an AI financial analyst tasked with generating a professional investment report summary. Using the provided mutual fund data, present the report in a client-friendly format where the client can choose the best mutual fund based on their needs. Ensure factual accuracy, regulatory compliance, and clarity in your explanation. Use the following structure:

Section 1: Mutual Fund Options Overview

Present a summary of the mutual fund options available, including their key performance metrics (e.g., returns, risk levels, focus areas).

Section 2: Client-Specific Comparisons

Compare the mutual funds against the client’s risk tolerance, investment goals, and preferences. Highlight how each fund aligns with these criteria.

Section 3: Tailored Recommendations

Suggest the most suitable mutual fund(s) for the client based on the comparisons, providing reasons for the recommendation.

Section 4: Disclosures and Compliance

Include mandatory regulatory disclosures and disclaimers to ensure compliance with industry standards.

Data Input:

Mutual Fund Data: [Provide detailed mutual fund performance metrics]

Compliance Rules: [e.g., Include disclaimers on past performance not guaranteeing future results.]
Mutual Fund Information: {question}
Context: {context}
Answer:
"""



PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Set up retriever (assuming 'langchain_chroma' is your vector store and is already initialized)
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 5})

# Function to remove duplicates from retrieved documents
def remove_duplicates(documents):
    seen = set()
    unique_docs = []
    for doc in documents:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    return unique_docs

# Set up the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def get_optimized_recommendations(query):
    # Retrieve documents based on the query
    raw_docs = retriever.get_relevant_documents(query)

    # Remove duplicates
    unique_docs = remove_duplicates(raw_docs)

    # Prepare the context from the documents
    context = " ".join([doc.page_content for doc in unique_docs])

    # Use the QA chain to get the response
    result = qa_chain({"context": context, "query": query})
    return result

# Example query
query = input("Please enter your query: ")
response = get_optimized_recommendations(query)
print(response)


Please enter your query: can we invest in aditya birla
{'context': 'Scheme Name: Aditya Birla SL Inter Equity – Plan A-Direct Growth, Min SIP: 1000, Min Lumpsum: 1000, Expense Ratio: 1.75%, Fund Size: 108.0 Cr, Fund Age: 10 years, Fund Manager: Vinod Narayan Bhat, Sortino: 1.28, Alpha: -, SD: 13.99, Beta: -, Sharpe: 0.81, Risk Level: 6, AMC: Aditya Birla Sun Life Mutual Fund, Rating: 0, Category: Equity, Sub-Category: Sectoral / Thematic Mutual Funds, 1 Yr Return: 0.9%, 3 Yr Return: 17.7%, 5 Yr Return: 10.4% Scheme Name: Aditya Birla SL Inter Equity – Plan B-Direct Growth, Min SIP: 1000, Min Lumpsum: 1000, Expense Ratio: 2.15%, Fund Size: 90.0 Cr, Fund Age: 10 years, Fund Manager: Vinod Narayan Bhat, Sortino: 2.45, Alpha: -1.93, SD: 16.3, Beta: 0.87, Sharpe: 1.14, Risk Level: 6, AMC: Aditya Birla Sun Life Mutual Fund, Rating: 0, Category: Equity, Sub-Category: Sectoral / Thematic Mutual Funds, 1 Yr Return: -4.1%, 3 Yr Return: 22.7%, 5 Yr Return: 7.6% Scheme Name: Aditya Birla SL Tax Re

In [ ]:
print(response['result'][response['result'].find("Answer:"):])



Answer:

Section 1: Mutual Fund Options Overview

The mutual fund options presented are Aditya Birla SL Inter Equity – Plan A-Direct Growth, Aditya Birla SL Inter Equity – Plan B-Direct Growth, Aditya Birla SL Tax Relief ’96 – Direct Growth. These funds have varying expense ratios, minimum SIP and lumpsum investments, and fund sizes. The Aditya Birla SL Inter Equity – Plan A-Direct Growth and Aditya Birla SL Inter Equity – Plan B-Direct Growth both have a Sortino ratio of 1.28, indicating a high return for the level of risk taken. However, the Aditya Birla SL Inter Equity – Plan B-Direct Growth has a higher alpha of -1.93, indicating a lower risk-adjusted return compared to the Aditya Birla SL Inter Equity – Plan A-Direct Growth, which has an alpha of -. The Aditya Birla SL Tax Relief ’96 – Direct Growth has a lower Sortino ratio of 1.94, indicating a lower return for the level of risk taken. The Aditya Birla SL Inter Equity – Plan A-Direct Growth and Aditya Birla SL Inter Equity – Pla